# Benchmark for bayesian optimization

## Imports

In [1]:
# !git clone https://github.com/abauville/Notes_Gaussian_processes.git
# !cp /content/Notes_Gaussian_processes/bayes_lib.py .
# !pip install gpytorch
# !pip install botorch
from datetime import datetime

from bayes_lib import ExactGPModel, train_hyper_params, get_x_new, run_experiment
import torch
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.constraints import Interval
import botorch
from botorch.test_functions.synthetic import Hartmann
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

## Define the ground truth, baseline and metric

In [3]:
def gt_func(x):
    """Ground truth function: negative hartmann 6
    The Bayes opt library we use aims to maximize functions by default.
    We use the negated function to effectively minimize it, i.e. argmin(f(x)) = argmax(-f(x))
    """
    hart = Hartmann()
    return - hart.evaluate_true(x)

def error_gap(current_best):
    """Returns the absolute difference between the global minimum of the hartmann 6 function
    and the current best value
            Error gap := |min f(x*) -  current best|
    """
    hart = Hartmann()
    return abs(current_best - (- hart.optimal_value))

def baseline_model(gt_func, n_iter, n):
    """A model that attempts to maximize by picking n random sample and keeping the minimum value. 
    The process is repated over n_iter iterations
    Parameters
    ============
    n_iter: int
        number of iterations
    n: int
        number of samples drawn at each iteration
        
    Returns
    ============
    array of shape (n_iter,)
        contains the max of the function discovered so far at each iteration
    """
    results = -np.ones(n_iter)
    results[0] = gt_func(torch.randn(n*6).reshape(-1,6)).max()

    for i in range(2, n_iter):
        results[i] = max(results[i-1], gt_func(torch.randn(n*6).reshape(-1,6)).max())
    return results

## Run the Bayesian optimization

### Initialize

In [4]:
n_exp = 10                  # number of experiments
n_iter = 200                # number of iterations
print_period = 5           # results are printed every print_period iteration
n_train_ini = 20            # number of initial training examples



error_gaps = np.zeros([n_exp, n_iter]) # array to store the metrics

### Optimization loop

In [6]:
from datetime import datetime
for i_exp in range(n_exp):
    print(f"Experiment {i_exp: 02d}")
    print( "=============")
    error_gaps = run_experiment(gt_func, n_iter, n_train_ini)
    
    # Save intermediate results
    # ============================
    timestamp = datetime.now().strftime("%Y%m%d%H%M%s")
    np.save("timestamp_error_gaps", error_gaps)
    

030/200: 3.18763, 3.05602, -0.87051


KeyboardInterrupt: 

'2022042521091650888590'

# Plot

In [ ]:
def plot_series(X, label=''):
    means = np.mean(X,0)
    stds = np.std(X,0)
    ax.plot(means, label=label)
    ax.fill_between(np.arange(n_iter), 
                    means + 2.0*stds, 
                    means - 2.0*stds, 
                    alpha=0.2)

In [ ]:
baseline_results = np.array([baseline_model(n_iter, int(1e3)) for i in range(n_exp)])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=[7,7])
plot_series(baseline_results, label='baseline')
plot_series(error_gaps, label='BayesOpt')
ax.set_yscale('log')
plt.legend()
plt.title("Hartmann 6D function minimization")
plt.ylabel("error gap")
plt.xlabel("func. evaluations")
plt.savefig("hartmann_min.png")